In [1]:
import pandas as pd
import numpy as np

from datetime import datetime

data_path = "...\\data\\raw\\COVID-19\\csse_covid_19_data\\csse_covid_19_time_series\\time_series_covid19_confirmed_global.csv"
pd_raw = pd.read_csv(data_path)
    
pd_data_base=pd_raw.rename(columns={'Country/Region':'country', 'Province/State':'state'})
pd_data_base['state']=pd_data_base['state'].fillna('no')
pd_data_base=pd_data_base.drop(['Lat','Long'],axis=1)
   
pd_relational_model=pd_data_base.set_index(['state','country'])\
                    .T\
                    .stack(level=[0,1])\
                    .reset_index()\
                    .rename(columns={'level_0':'date', 0:'confirmed'},)
    
pd_relational_model['date']=pd_relational_model.date.astype('datetime64[ns]')
   
pd_relational_model.to_csv('...\\data\\processed\\COVID_relational_confirmed.csv' , sep=';', index=False)


In [2]:
pd_JH_data=pd.read_csv('...\\data\\processed\\COVID_relational_confirmed.csv' , sep=';', parse_dates=[0])
pd_JH_data=pd_JH_data.sort_values('date',ascending=True).reset_index(drop=True).copy()
pd_JH_data.head()

,date,state,country,confirmed
0,2020-01-22,Alberta,Canada,0.0
1,2020-01-22,no,"Korea, South",1.0
2,2020-01-22,no,Kosovo,0.0
3,2020-01-22,no,Kuwait,0.0
4,2020-01-22,no,Kyrgyzstan,0.0


In [3]:
test_data=pd_JH_data[((pd_JH_data['country']=='India')|
                     (pd_JH_data['country']=='Germany'))&
                    (pd_JH_data['date']>'2020-03-20')&
                    (pd_JH_data['date']<'2020-03-29')]

In [4]:
test_data

,date,state,country,confirmed
15737,2020-03-21,no,Germany,22213.0
15749,2020-03-21,no,India,330.0
16002,2020-03-22,no,Germany,24873.0
16015,2020-03-22,no,India,396.0
16269,2020-03-23,no,Germany,29056.0
16281,2020-03-23,no,India,499.0
16535,2020-03-24,no,Germany,32986.0
16548,2020-03-24,no,India,536.0
16801,2020-03-25,no,Germany,37323.0
16813,2020-03-25,no,India,657.0


In [5]:
test_data.groupby(['country']).agg(np.max)

,date,state,confirmed
country,,,
Germany,2020-03-28,no,57695.0
India,2020-03-28,no,987.0


In [6]:
test_data.groupby(['state','country']).agg(np.max)

date  confirmed
state country                      
no    Germany 2020-03-28    57695.0
      India   2020-03-28      987.0

In [7]:
from sklearn import linear_model

reg = linear_model.LinearRegression(fit_intercept=True)

def get_doubling_time_via_regression(in_array):
    
    y= np.array(in_array)
    X= np.arange(-1,2).reshape(-1, 1)
    
    assert len(in_array)==3
    
    reg.fit(X,y)
    intercept = reg.intercept_
    slope = reg.coef_
    
    return intercept/slope

In [8]:
def rolling_reg(df_input,col='confirmed'):
    
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression, raw=False)
    return result

In [9]:
test_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed')

state  country       
no     Germany  15737         NaN
                16002         NaN
                16269    7.417994
                16535    7.142035
                16801    8.012983
                17066    6.954407
                17333    6.501919
                17598    7.390371
       India    15749         NaN
                16015         NaN
                16281    4.832347
                16548    6.814286
                16813    7.139241
                17079    6.701571
                17346    6.582609
                17611    6.669231
Name: confirmed, dtype: float64

In [10]:
pd_DR_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed')
pd_DR_result=pd_DR_result.reset_index()

In [11]:
pd_DR_result=pd_DR_result.rename(columns={'confirmed':'doubling_rate',
                            'level_2':'index'})
pd_DR_result.head()

,state,country,index,doubling_rate
0,Alberta,Canada,0,NaN
1,Alberta,Canada,465,NaN
2,Alberta,Canada,701,NaN
3,Alberta,Canada,998,NaN
4,Alberta,Canada,1231,NaN


In [12]:
pd_JH_data=pd_JH_data.reset_index()

In [13]:
pd_result_larg=pd.merge(pd_JH_data,pd_DR_result[['index','doubling_rate']],on=['index'],how='left')

In [14]:
pd_result_larg

,index,date,state,country,confirmed,doubling_rate
0,0,2020-01-22,Alberta,Canada,0.0,NaN
1,1,2020-01-22,no,"Korea, South",1.0,NaN
2,2,2020-01-22,no,Kosovo,0.0,NaN
3,3,2020-01-22,no,Kuwait,0.0,NaN
4,4,2020-01-22,no,Kyrgyzstan,0.0,NaN
5,5,2020-01-22,no,Laos,0.0,NaN
6,6,2020-01-22,no,Latvia,0.0,NaN
7,7,2020-01-22,no,Lebanon,0.0,NaN
8,8,2020-01-22,no,Lesotho,0.0,NaN
9,9,2020-01-22,no,Liberia,0.0,NaN


# Filtering the data with groupby apply

In [15]:
from scipy import signal

def savgol_filter(df_input,column='confirmed',window=5):
    window=5,
    degree=1
    df_result=df_input
    
    filter_in=df_input[column].fillna(0)
    
    result=signal.savgol_filter(np.array(filter_in),
                                5,
                                1)
    
    df_result[column + '_filtered']=result
    return df_result

In [16]:
pd_filtered_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(savgol_filter)
pd_filtered_result=pd_filtered_result.reset_index()

C:\Users\Admin\Anaconda3\lib\site-packages\scipy\signal\_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


In [17]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_result[['index','confirmed_filtered']],on=['index'],how='left')
pd_result_larg.head()

,index,date,state,country,confirmed,doubling_rate,confirmed_filtered
0,0,2020-01-22,Alberta,Canada,0.0,NaN,0.0
1,1,2020-01-22,no,"Korea, South",1.0,NaN,0.8
2,2,2020-01-22,no,Kosovo,0.0,NaN,0.0
3,3,2020-01-22,no,Kuwait,0.0,NaN,0.0
4,4,2020-01-22,no,Kyrgyzstan,0.0,NaN,0.0


In [18]:
pd_filtered_doubling=pd_result_larg[['state','country','confirmed_filtered']].groupby(['state','country']).apply(rolling_reg,'confirmed_filtered').reset_index()

pd_filtered_doubling=pd_filtered_doubling.rename(columns={'confirmed_filtered':'doubling_rate_filtered',
                                                        'level_2':'index'})

pd_filtered_doubling.head()

,state,country,index,doubling_rate_filtered
0,Alberta,Canada,0,NaN
1,Alberta,Canada,465,NaN
2,Alberta,Canada,701,NaN
3,Alberta,Canada,998,NaN
4,Alberta,Canada,1231,NaN


In [19]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_doubling[['index','doubling_rate_filtered']],on=['index'],how='left')
pd_result_larg.head()

,index,date,state,country,confirmed,doubling_rate,confirmed_filtered,doubling_rate_filtered
0,0,2020-01-22,Alberta,Canada,0.0,NaN,0.0,NaN
1,1,2020-01-22,no,"Korea, South",1.0,NaN,0.8,NaN
2,2,2020-01-22,no,Kosovo,0.0,NaN,0.0,NaN
3,3,2020-01-22,no,Kuwait,0.0,NaN,0.0,NaN
4,4,2020-01-22,no,Kyrgyzstan,0.0,NaN,0.0,NaN


In [20]:
mask=pd_result_larg['confirmed']>100
pd_result_larg['doubling_rate_filtered']=pd_result_larg['doubling_rate_filtered'].where(mask, other=np.NaN)

In [21]:
pd_result_larg.to_csv('...\\data\\processed\\COVID_final_set.csv' , sep=';' , index=False)